### Model
| model |
| --- |
| gpt-3.5-turbo-0125 |
| gpt-4-0125-preview |

In [6]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1
)

In [9]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Korean"),
    AIMessage(content="안녕하세요. 저는 AiGenda 봇입니다."),
    HumanMessage(content="서울과 도쿄의 거리는 얼마인가요. 너의 이름은 뭐니?"),
]

chat.predict_messages(messages)

AIMessage(content='서울과 도쿄의 거리는 약 1,200km 정도입니다. 제 이름은 AiGenda입니다. 더 궁금한 점이 있으시면 언제든지 물어보세요.')